In [1]:
class Node:
    def __init__(self,NodeName,NodeType):
        self.NodeName = NodeName
        self.NodeType = NodeType
        self.childrenList = []
    
        
        

In [2]:
import math
def calculateClassInfoGain(labels):
    sset = set(labels)
    counts = []
    #print(labels)
    for item in sset:
        counts.append(labels.count(item))
    
    I_p_n = -((counts[0]/(counts[0]+counts[1]))*math.log2((counts[0]/(counts[0]+counts[1]))))-((counts[1]/(counts[0]+counts[1]))*math.log2((counts[1]/(counts[0]+counts[1]))))
    return I_p_n

In [3]:
def printAnswer(answer):
    #print('#########################')
    print(answer.NodeName)
    for entry in answer.childrenList:
        if entry != 'None':
            print(entry[0],'         ')
       
    
    for entry in answer.childrenList:
         print(printAnswer(entry[1]))
        
    #print('#########################')
        

In [4]:
def divideData(data,unique_attribute_values,index,n):
    #print(data)
    #print('****************',index,'****************')
    NEW = []
    
    for value in unique_attribute_values:
        print(value)
        new_data = []
        n = len(data[0])
        cbp = data[0]
        cbp = cbp[0:index]+cbp[index+1:n]
        new_data.append(cbp)
        
        for i in range(1,len(data)):
            #print(data[i][index])
            if data[i][index] == value:
                
                empty = []
                for j in range(0,index):
                    empty.append(data[i][j])
                for j in range(index+1,n):
                    empty.append(data[i][j])
                    
                new_data.append(empty)
        
        NEW.append(new_data)
        
    
    return NEW

In [5]:
def calculateInformationGain(attrName,attr_list,attr_labels,P,N,cig):
    set_of_labels = set(attr_labels)
    #as of now it is guarenteed that class-label is binary(1/0) so definitely |set_of_labels| == 2
    
    Set = set(attr_list)
    
    table = []
    dictionary = {}
    i=0
    for attr_value in Set:
        l = [attr_value,0,0]
        table.append(l)
        dictionary[attr_value] = i
        i=i+1
     
    j=0
    for x in attr_list:
        index = dictionary[x]
        if attr_labels[j] == P:
            table[index][1]+=1
        else:
            table[index][2]+=1
        j+=1
    
    #print(table)
    
    number = len(attr_labels)
    
    entropy = 0
    for listt in table:
        Ipn = 0
        if listt[1] == 0 or listt[2]==0:
            Ipn = 0
        elif listt[1] == listt[2]:
            Ipn = 1
        else:
            Ipn = -((listt[1]/(listt[1]+listt[2]))*math.log2((listt[1]/(listt[1]+listt[2]))))-((listt[2]/(listt[1]+listt[2]))*math.log2((listt[2]/(listt[1]+listt[2]))))
        
        entropy += ((listt[2]+listt[1])/number) * Ipn
    
    gain = cig - entropy
    return gain
    
    
    
    

In [6]:
#initially root is null 
root = None

#Initially every candidate is potential candidate for being root


#initial call will be DecisionTree(None,All attributes,Original Data)
def DecisionTree(parent,data):
    Attribute_List = data[0]
    Attribute_List = Attribute_List[:len(Attribute_List)-1]
    
    #print('*****************************start****************************')
   
    #print(Attribute_List)
    #print(data)
    #last column is class-label
    
    #base case 1:attribute list is empty then return a Node with name = class_label where frequency of class label is more
    if len(Attribute_List) == 0:
        #print(data)
        
        label_list = []
        for i in range(1,len(data)):
            label_list.append(data[i][-1])
        
        #print(label_list)
        
        
        set_of_labels = set(label_list)
        count = -1
        labell = None
        for label in set_of_labels:
            if label_list.count(label) > count:
                count = label_list.count(label)
                labell = label
                
        obj = Node(labell,'label')
        #print('***************************end******************************')
        return obj
    
    #base case2 : #every tuple has same class label so create object of Node class with NodeName = that classlabel and simply return
    flag = 0
    #print('length == ',len(data))
    for i in range (2,len(data)):
        #print(data[i][-1])
        if data[i][-1] == data[i-1][-1]:
            continue
        else:
            flag=1
            break
        
    if flag == 0:
        obj = Node(data[1][-1],'label')
        #print('*********************************************************')
        return obj
    
    
    
    labels = []
    for i in range(1,len(data)):
        labels.append(data[i][-1])
    
    classInfoGain = calculateClassInfoGain(labels)
    
    #else calculate InfoGain for every attribute and then select the one with max InfoGain
    infoGain = -1
    Attr = -1
    for i in range(len(Attribute_List)):
        attr_list =[]
        attr_labels = []
        for j in range(1,len(data)):
            attr_list.append(data[j][i]) 
            attr_labels.append(data[j][-1])
        
        b = calculateInformationGain(Attribute_List[i],attr_list,attr_labels,'Y','N',classInfoGain)
        #print('b = ',b)
        if b>infoGain:
            infoGain = b
            Attr = i
        
    #print(infoGain)
    
    print('SELECTED = ',Attribute_List[Attr])
    
    current_root = Node(Attribute_List[Attr],'internal node')
    
    
    
    k = []
    for i in range(1,len(data)):
        k.append(data[i][Attr])
        
    unique = set(k)
    
    datasets = divideData(data,unique,Attr,len(Attribute_List))
    
    
    
   
    for member in datasets:
        #print('**********************************************************')
        cd = unique.pop()
        #print(cd)
        #print(member)
       
        obj = DecisionTree(current_root,member)
        club = (cd,obj)
        current_root.childrenList.append(club)
        
        
    #print('*********************************************************')
        
    #printAnswer(current_root.childrenList)
    
    
    return current_root
        

In [7]:
# if __name__=="__main__":
    #read data from csv file
import csv

filename = "C:\\Users\\Amruta K\\Downloads\\transaction - Sheet1 (2).csv"
data = csv.reader(open(filename,"rt"))
dataset = list(data)
    #print(dataset)
answer = DecisionTree(None,dataset)
print('************************************************')
printAnswer(answer)

SELECTED =  Age
>35
21-35
<21
SELECTED =  Marital Status
Single
Married
SELECTED =  Gender
M
F
************************************************
Age
>35          
21-35          
<21          
Marital Status
Single          
Married          
Y
None
N
None
None
Y
None
Gender
M          
F          
N
None
Y
None
None
